<h1>RadxRate Tutorial</h1>

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local PID category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

<h2>Overview</h2>


**1. Parameter files**
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (Radar d
ependent)
* Grib2toMdv params

### **Parameter files modifications**
* **RadxRate main params**
    * input_dir (radar data) [line 105]
    * INPUT FIELD INFORMATION. [line 155]
        * SNR_available = TRUE; change to FALSE; [line 177]
        * DBZ_field_name = "DBZ"; change to "REF"; [line 215]
        * LDR_available = TRUE; change to FALSE [line 255]
    * kdp_params_file_path  [line 284]
    * pid_params_file_path  [line 303]
    * PID_use_attenuation_corrected_fields = TRUE [line 331]
    * RATE_params_file_path [line 350]
    * RATE_use_attenuation_corrected_fields = TRUE [line 378]
    * output_dir [line 699] -- or set in command line with -outdir flag
    * output_format = OUTPUT_FORMAT_CFRADIAL; (Default) [line 755] 
* **RadxRate Kdp_specific params**
    * KDP_fir_filter_len = KDP_FIR_LEN_10; [line 65]
    * KDP_psob_method = PEAK_REMOVAL_METHOD; [line 114]
* **RadxRate Pid_specific params**
    * PID_thresholds_file_path [line 27]
    * PID_use_soundings_from_spdb = FALSE; set to TRUE[line 303]
    * PID_sounding_spdb_url = "spdb/soundings"; [line 313]
    * PID_sounding_location_name [line 339]
    *  **do sounding steps first before filling Pid variables out**
* **RadxRate Rate_specific params**
    * N/A?
* **Pid Thresholds params**
    * Radar dependent
        * Nexrad is S-band simultaneous h/v transmission and recieving


https://stackoverflow.com/questions/19699059/print-directory-file-structure-with-icons-for-representation-in-markdown
```
BASE_DIR
│   RadxRate_tutorial.ipynb
│ 
│
└───data
│   │   ...
│   │   
│   │
│   │───raw
│   │   │   KMHX20180914_191822_V06.ar2v
│   │   │   gfsanl_4_2018091418.g2.tar
│   │   │   
│   │───CfRadial
│   │   │   ...
│   │   │
│   │   │
│   │───sounding 
│   │   │   ...
│   │   │
│    ───
│   
└───params
│   │   Pid_thresholds.sband.shv.txt
│   │   RadxRate_main_params
│   │   RadxRate_Kdp_params
│   │   RadxRate_Pid_params
│   │   RadxRate_rate_params



- Print out parameter files

In [1]:
# # create original parameter files
# !/usr/local/lrose/bin/RadxRate -print_params > ./params_ori/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params_ori/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params_ori/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params_ori/RadxRate_rate_params

# # setup parameter files for Users
# !/usr/local/lrose/bin/RadxRate -print_params > ./params/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params/RadxRate_rate_params
!/usr/local/lrose/bin/Grib2toMdv -print_params > ./params/Grib2toMdv_params

Program 'Grib2toMdv'
Run-time 2022/12/17 22:34:03.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to c

In [2]:
import os

In [29]:
os.environ['BASE_DIR'] = '/home/jovyan/RadxRate_tutorial'
os.environ['RADAR_NAME'] = 'KMHX'
!echo "Base directory: "$BASE_DIR
!echo "Base directory: "$RADAR_NAME

Base directory: /home/jovyan/RadxRate_tutorial
Base directory: KMHX


# Data download and directory organization

https://drive.google.com/drive/folders/1nQ1-CUR_OFuSdWgsE-qL35L5rcWTmEh4?usp=share_link

In [4]:
### make directory for pre-made parameter files
# !rm -rf ${BASE_DIR}/params
# !mkdir ${BASE_DIR}/params

### make a directory for all the data files (raw and analysis)
# !rm -rf ${BASE_DIR}/data
# !mkdir ${BASE_DIR}/data

### make subdirectory within data for the raw data
# !rm -rf ${BASE_DIR}/data/raw
# !mkdir ${BASE_DIR}/data/raw

### make subdirectory within data for sounding data
# !rm -rf ${BASE_DIR}/data/sounding
# !mkdir ${BASE_DIR}/data/sounding

### wget raw files from ftpserver and plop in $BASE_DIR
# wget ftpserver/KMHX20180914_191822_V06.ar2v
# wget ftpserver/gfsanl_4_2018091418.g2.tar

### move files to proper directories
# !mv KMHX20180914_191822_V06.ar2v ${BASE_DIR}/data/raw
# !mv gfsanl_4_2018091418.g2.tar ${BASE_DIR}/data/raw

# Convert Nexrad Level II to CfRadial files

In [17]:
# !rm -rf ${BASE_DIR}/data/CfRadial
# !mkdir ${BASE_DIR}/data/CfRadial
!/usr/local/lrose/bin/RadxConvert -f ${BASE_DIR}/data/raw/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/CfRadial

Program 'RadxConvert'
Run-time 2022/12/15 18:51:20.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to 

# Untar GFS analysis grib2 files for sounding
* Grb2 --> Mdv --> Spdb

In [28]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/sounding/grib
!mkdir ${BASE_DIR}/data/sounding/grib

# extract file from tape archive (tar)
!tar -xvf ${BASE_DIR}/data/raw/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/sounding/grib/

gfs_4_20180914_1800_000.grb2
gfs_4_20180914_1800_003.grb2
gfs_4_20180914_1800_006.grb2


- **Grib2toMdv**
    - input_dir [line 69] optional, if directory is specified in command line
    - write_non_forecast [line 385]
    - non_forecast_mdv_url (location of outputted mdv files) [line 395]
    - out_projection_info [line 715] Radar location 
    - out_grid_info [line 749] Subjective box around radar(s)
    - height_levels [line 838] Height levels, optional

In [11]:
# convert model analysis to Mdv
!/usr/local/lrose/bin/Grib2toMdv -params ${BASE_DIR}/params/Grib2toMdv_params -f ${BASE_DIR}/data/sounding/grib/*.grb2

Program 'Grib2toMdv'
Run-time 2022/12/17 23:38:23.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to c

- **Mdv2SoundingSpdb**
    - input data field names [line 126] 
    - sampling locations [line 264]
        - Need to input location of current radar KMHX
    - ouput_url [line 363]

In [27]:
# convert Mdv to SoundingSpdb
!/usr/local/lrose/bin/Mdv2SoundingSpdb -params ${BASE_DIR}/params/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/sounding/mdv/20180914/*.mdv*
!/usr/local/lrose/bin/Mdv2SoundingSpdb -params ${BASE_DIR}/params/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/sounding/mdv/20180915/*.mdv*

Program 'Mdv2SoundingSpdb'
Run-time 2022/12/18 00:37:22.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as no

- Check sounding data is filled (except vertical velocity and divergence)

In [28]:
!/usr/local/lrose/bin/SpdbQuery -url ${BASE_DIR}/data/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"

url: /home/jovyan/RadxRate_tutorial/data/sounding/spdb
Prod label: Sounding Data
Prod id:    301
DataType: 0
N Chunks:   2

Header for chunk ---> 0 <---
  data_type: 1481133387 (KMHX)
  data_type2: 0
  valid_time: 2018/09/14 18:00:00
  expire_time: 2018/09/15 06:00:00
  write_time: 2022/12/18 00:37:22
  len: 1448
  current_compression: none
  stored_compression: none


Data for sounding:
Sounding Product
  launch time      = 2018/09/14 18:00:00
  number of points = 29
  sourceId         = 3
  leadSecs         = 0
  spare int[0] = 0
  spare int[1] = 0
  lat = 34.775902
  lon = -76.876198
  alt = 43.000000
  missing = -9999.000000
  spare float[0] = 0.000000
  spare float[1] = 0.000000
  sourceName = RUC
  sourceFmt  = Mdv2SoundingSpdb
  siteName   = KMHX
  Sounding Point 0
    pressure = 1000.000000
    altitude = 0.000000
    u  wind  = -15.068794
    v  wind  = 9.757457
    w  wind  = -9999.000000
    relHum   = 84.960449
    temp     = 25.810978
    div      = -9999.000000
    spare[

# Setup steps complete, run RadxRate

In [35]:
!/usr/local/lrose/bin/RadxRate -params ${BASE_DIR}/params/RadxRate_main_params -f ${BASE_DIR}/data/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/rate